# Content Editor Agent
### Agent uses tools and user instruction to update some content

### 1) Install Packages

In [ ]:
#!pip install strands-agents strands-agents-tools
#!pip install tavily-python
#https://pypi.org/project/tavily-python/

### 2) Setup API Keys, Import Packages and initiate model
#### Using OpenAI model 
#### Using Tavily for internet based search
#### Learn about creating Tavily API Key - https://www.tavily.com/blog/getting-started-with-the-tavily-search-api

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'
os.environ["TAVILY_API_KEY"] ='YOUR_TAVILY_API_KEY'

In [ ]:
from strands import Agent, tool
from strands.models.openai import OpenAIModel
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [ ]:
# Create a Model
model = OpenAIModel(
    model_id="gpt-5-nano"
)

### 3) Initial Content
#### This is the content which you want to improve using agent. In this example, we are passing a sample executive summary.
#### It could also be content generated by another agent or LLM

In [ ]:
input_exec_summary = """
Executive Summary
Digitization in banks is reshaping delivery, operations, and risk management. Core modernization, expanded digital channels, cloud adoption, and APIs enable real-time payments, personalized experiences, and scalable, cost-efficient operations. Benefits include faster onboarding, higher conversion, improved underwriting and risk controls, and greater resilience. Realizing value requires disciplined governance, robust cybersecurity, regulatory alignment, and talent equipped for a digital-first environment.

Key Drivers and Capabilities
- Real-time payments and instant onboarding; open banking APIs for ecosystem partnerships
- Cloud platforms designed with security by default
- AI/ML for fraud detection, KYC, credit scoring, and personalization
- Data governance, privacy, and analytics to inform decisions

Impact and Benefits
- Enhanced customer experience, higher retention, faster cycle times
- Lower manual work, reduced costs, and improved efficiency
- Strengthened risk management and compliance through analytics
- New revenue through digital products and embedded finance

Risks and Considerations
- Legacy architectures, data silos, and integration complexity
- Cybersecurity, data privacy, and regulatory compliance
- Talent gaps, change management, and vendor risk

Strategic Recommendations
- Define a customer-centric digital strategy with a modular core
- Pursue secure cloud adoption and a robust data governance framework
- Partner with fintechs; maintain strong governance and risk monitoring
- Invest in people, culture, and continuous digital-risk management

"""

### 4) Configure Agent
#### Conversation manager to keep last 10 messages in the agent memory.
#### Adding tavilty search as tool for the agent to search and fetch information over internet
#### Putting executive summary as the first message for the agent

In [ ]:
# Tavily API for any internet based search
@tool
def search_tavily(query):
    """Search query over internet to fetch results
    Args:
        query: query which is searched over internet
    """
    tavily_client = TavilyClient()
    response = tavily_client.search(query)
    output = ""
    count = 1
    for r in response["results"]:
        output = output +  "{0}- Content: {1}. Source: {2} \n".format(count, r["content"], r["url"])
        count = count + 1
    return output

In [ ]:
# System message for the agent
exec_summ_improve_system_prompt = f"""
You are a helpful assistant which can improve executive summary based on user feedback.

Available Tools:
 - search_tavily: Use this tool to search the internet for up-to-date, reliable information relevant to the user feedback.
        
Update executive summary strickly based on user feedback. Do not add anything additional.
Improve executive summary without any further questions or comment."""

In [ ]:
# Configure conversation manager
conversation_manager = SlidingWindowConversationManager(window_size=10)

# Configure agent
exec_summ_improve_agent = Agent(
    model=model, 
    system_prompt=exec_summ_improve_system_prompt, 
    conversation_manager=conversation_manager,
    tools=[search_tavily],
    messages=[{"role": "user", "content": [{"text": "Here is the executive summary: " + input_exec_summary}]}
])

### 5) Run Agent
#### Provide user input to change executive summary content
#### You can run next cell multiple times with different user input to see agent updating the content
#### Few sample user inputs brlow
- Add a summary section in the end
- Add a new section Latest News with latest news from the internet about the topic
- Search latest information about risks and appened more details
- Change section content in paragraph in place of bullet points
- Translate to french 

In [ ]:
user_input = "Translate to french"
response = exec_summ_improve_agent(user_input)

In [ ]:
# Print updated executive summary
exec_summary = response.message["content"][0]["text"]
print(exec_summary)